# 01. 문자 단위 RNN(Char RNN)


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [3]:
input_str = 'apple'
label_str = 'pple!'
char_vocab = sorted(list(set(input_str+label_str)))
print(char_vocab)
vocab_size = len(char_vocab)
print ('문자 집합의 크기 : {}'.format(vocab_size))

['!', 'a', 'e', 'l', 'p']
문자 집합의 크기 : 5


In [4]:
input_size = vocab_size # 입력의 크기는 문자 집합의 크기
hidden_size = 5
output_size = 5
learning_rate = 0.1

In [9]:
for i, c in enumerate(char_vocab):
    print('인덱스 : ', i)
    print('list안의 요소 : ', c)

인덱스 :  0
list안의 요소 :  !
인덱스 :  1
list안의 요소 :  a
인덱스 :  2
list안의 요소 :  e
인덱스 :  3
list안의 요소 :  l
인덱스 :  4
list안의 요소 :  p


In [10]:
char_to_index = dict((c, i) for i, c in enumerate(char_vocab)) # 문자에 고유한 정수 인덱스 부여
print(char_to_index)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


 나중에 예측 결과를 다시 문자 시퀀스로 보기위해서 반대로 정수로부터 문자를 얻을 수 있는 index_to_char을 만든다.

In [15]:
index_to_char={}
for key, value in char_to_index.items():
    print('key : ',key)
    print('value : ',value)
    index_to_char[value] = key
print(index_to_char)

key :  !
value :  0
key :  a
value :  1
key :  e
value :  2
key :  l
value :  3
key :  p
value :  4
{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [17]:
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print(x_data) # 'apple'
print(y_data) # 'pple!'

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


파이토치의 nn.RNN()은 기본적으로 3차원 텐서를 입력받습니다. 그렇기 때문에 배치 차원을 추가해줍니다.

In [18]:
# 배치 차원 추가
# 텐서 연산인 unsqueeze(0)를 통해 해결할 수도 있었음.
x_data = [x_data]
y_data = [y_data]
print(x_data)
print(y_data)

[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]


In [23]:
np.eye(5)

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [19]:
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print(x_one_hot)

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


입력 데이터와 레이블 데이터를 텐서로 바꿔줍니다.

In [28]:
X = torch.FloatTensor(x_one_hot) # 실수데이터
Y = torch.LongTensor(y_data) # 정수데이터

In [27]:
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([1, 5, 5])
레이블의 크기 : torch.Size([1, 5])


In [29]:
class Net(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True) # RNN 셀 구현
        self.fc = torch.nn.Linear(hidden_size, output_size, bias=True) # 출력층 구현

    def forward(self, x): # 구현한 RNN 셀과 출력층을 연결
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [30]:
net = Net(input_size, hidden_size, output_size)

In [31]:
outputs = net(X)
print(outputs.shape) # 3차원 텐서

torch.Size([1, 5, 5])


(1, 5, 5)의 크기를 가지는데 각각 배치 차원, 시점(timesteps), 출력의 크기입니다. 나중에 정확도를 측정할 때는 이를 모두 펼쳐서 계산하게 되는데, 이때는 view를 사용하여 배치 차원과 시점 차원을 하나로 만듭니다.

In [32]:
print(outputs.view(-1, input_size).shape) # 2차원 텐서로 변환

torch.Size([5, 5])


In [33]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([1, 5])
torch.Size([5])


In [34]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [35]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, input_size), Y.view(-1)) # view를 하는 이유는 Batch 차원 제거를 위해
    loss.backward() # 기울기 계산
    optimizer.step() # 아까 optimizer 선언 시 넣어둔 파라미터 업데이트

    # 아래 세 줄은 모델이 실제 어떻게 예측했는지를 확인하기 위한 코드.
    result = outputs.data.numpy().argmax(axis=2) # 최종 예측값인 각 time-step 별 5차원 벡터에 대해서 가장 높은 값의 인덱스를 선택
    result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

0 loss:  1.7356224060058594 prediction:  [[1 1 0 0 1]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  aa!!a
1 loss:  1.4855117797851562 prediction:  [[4 4 4 4 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppppp
2 loss:  1.229673147201538 prediction:  [[4 4 3 3 3]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pplll
3 loss:  1.0192875862121582 prediction:  [[4 4 3 2 3]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pplel
4 loss:  0.796370804309845 prediction:  [[4 4 3 2 3]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pplel
5 loss:  0.5958911180496216 prediction:  [[4 4 3 2 3]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pplel
6 loss:  0.45572495460510254 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
7 loss:  0.3599584400653839 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
8 loss:  0.2905360758304596 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
9 loss:  0.23241467773914337 prediction:  [[4 4 3 2 0]] 